<a href="https://colab.research.google.com/github/dolmani38/similarly/blob/master/Text%20Generation%20With%20LSTM%20Recurrent%20Neural%20Networks%20in%20Python%20with%20Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text Generation With LSTM Recurrent Neural Networks in Python with Keras

https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [52]:
# load ascii text and covert to lowercase
target_url = "https://raw.githubusercontent.com/dolmani38/similarly/master/Sentance_dataset.txt"
import urllib.request
raw_text = urllib.request.urlopen(target_url).read().decode('utf-8')
raw_text = raw_text.lower()

In [53]:
raw_text


'울 성북구 사랑제일교회에서 지난 12일 신종 코로나바이러스 감염증(코로나19) 첫 확진자가 나온 지 8일 만인 20일까지 이 교회 관련 확진자가 700명이 넘게 발생한 것으로 파악됐다.\r\n\r\n특히 서울뿐 아니라 12개 시·도에서 확진자가 나왔고 여러 시설로 \'n차 전파\'가 이뤄지는 양상을 보이고 있다.\r\n\r\n김강립 중앙재난안전대책본부(중대본) 1총괄조정관은 21일 정부세종청사에서 열린 정례 브리핑에서 "20일 오후 6시 기준으로 이 교회와 관련해 총 3천415명을 검사했고 이 가운데 확진자는 총 739명"이라고 밝혔다.\r\n\r\n김 1총괄조정관은 이 교회와 관련해서 "현재까지 12개 시·도에서 환자 발생이 신고되고 있어 전국적인 전파로 이어지는 양상이고 직장, 의료기관, 다른 종교시설 등 총 18개 시설에서 추가전파가 확인되고 있다"고 우려했다.\r\n\r\n그러나 사랑제일교회에 대한 역학조사는 원활하게 이뤄지지 않고 있다는 게 정부의 설명이다. 역학조사를 통해 조기에 확진자와 접촉자를 확인할 수 없으면 환자 규모는 더 커질 수 있다.\r\n\r\n김 1총괄조정관은 "보다 정확한 교인 명단을 확보하기 위해 어제(20일)와 오늘(21일) 서울시, 경찰청과 합동으로 사랑제일교회에 대한 추가 역학조사를 진행했으나 교회 측의 비협조로 추가 자료를 확보하지 못했다"면서 "이에 이 교회 관계자들을 고발했고 수사기관의 압수수색 등을 통해 자료 확보를 추진할 계획"이라고 말했다.\r\n\r\n이어 그는 "(교회 관련자들이) 검사와 격리에 불응하는 경우가 있어 방역의 애로가 커지고 있는데 이런 상황은 법에 따라 엄정한 조치를 할 것"이라며 "역학조사 방해 등의 경우에는 지방자치단체가 손해배상 청구 등을 적극적으로 시행할 것"이라고 경고했다.\r\n\r\n중대본에 따르면 당국의 검사에 불응할 경우 벌금 200만 원, 자가격리를 위반할 경우 1년 이하 징역이나 1천만 원 이하의 벌금 처분을 받을 수 있다.\r\n한편 정부는 지난 15일 광화문 집회와 관련해

In [54]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [55]:
chars

['\n',
 '\r',
 ' ',
 '!',
 '"',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '<',
 '>',
 '?',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '~',
 '±',
 '·',
 'δ',
 '‘',
 '’',
 '“',
 '”',
 '…',
 '→',
 '∙',
 '∼',
 '▲',
 '△',
 '○',
 'だ',
 '㎜',
 '㎢',
 '一',
 '丈',
 '三',
 '下',
 '事',
 '二',
 '亘',
 '人',
 '介',
 '伊',
 '住',
 '佐',
 '作',
 '俊',
 '信',
 '倍',
 '偉',
 '光',
 '全',
 '前',
 '博',
 '大',
 '太',
 '夫',
 '安',
 '小',
 '山',
 '岸',
 '州',
 '市',
 '平',
 '幹',
 '恵',
 '慶',
 '應',
 '敏',
 '文',
 '早',
 '晋',
 '木',
 '東',
 '森',
 '榮',
 '橋',
 '正',
 '歷',
 '河',
 '淳',
 '渕',
 '湛',
 '無',
 '生',
 '田',
 '石',
 '竹',
 '義',
 '芳',
 '苗',
 '茂',
 '菅',
 '萩',
 '藤',
 '裕',
 '郎',
 '野',
 '階',
 '雄',
 '難',
 '青',
 '題',
 '高',
 '麻',
 '黨',
 '가',
 '각',
 '간',
 '갈',
 '감',
 '갑',
 '값',
 '갔',
 '강',
 '갖',
 '같',
 '개',
 '객',
 '갤',
 '거',
 '걱',
 '건',
 '걸',
 '검',
 '겁',
 '것',
 '

In [56]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  69850
Total Vocab:  991


In [57]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  69750


In [58]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [60]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256*4, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 1024)              4202496   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 991)               1015775   
Total params: 5,218,271
Trainable params: 5,218,271
Non-trainable params: 0
_________________________________________________________________


In [45]:
#Larger LSTM Recurrent Neural Network

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 65)                16705     
Total params: 806,209
Trainable params: 806,209
Non-trainable params: 0
_________________________________________________________________


In [61]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [62]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
545/545 [==============================] - ETA: 0s - loss: 4.9642
Epoch 00001: loss improved from inf to 4.96421, saving model to weights-improvement-01-4.9642.hdf5
545/545 [==============================] - 118s 216ms/step - loss: 4.9642
Epoch 2/20
545/545 [==============================] - ETA: 0s - loss: 4.7318
Epoch 00002: loss improved from 4.96421 to 4.73176, saving model to weights-improvement-02-4.7318.hdf5
545/545 [==============================] - 118s 216ms/step - loss: 4.7318
Epoch 3/20
545/545 [==============================] - ETA: 0s - loss: 4.4742
Epoch 00003: loss improved from 4.73176 to 4.47422, saving model to weights-improvement-03-4.4742.hdf5
545/545 [==============================] - 118s 217ms/step - loss: 4.4742
Epoch 4/20
545/545 [==============================] - ETA: 0s - loss: 4.3242
Epoch 00004: loss improved from 4.47422 to 4.32420, saving model to weights-improvement-04-4.3242.hdf5
545/545 [==============================] - 118s 217ms/step - l

In [63]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print(''.join([int_to_char[value] for value in pattern]))

Seed:
당원이 참석한 일 등에 재기된 통합당 책임론은 방역 실패 책임을 모면하기 위한 정치 공세라고 규정했다.
 
김종인 미래통합당 비상대책위원장은 21일 오전 국회에서 열린 시·도당


In [69]:
import sys
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm, 으산(8월 3일) 72.0mm